In [1]:
import requests
import copy
import csv
import time
import json
import uuid
import pprint
import asyncio
from typing import List
from urllib import parse
from urllib.parse import quote
import inspect
import datetime

In [2]:
class magic_fstring_function:
    def __init__(self, payload):
        self.payload = payload
        self.cached = None
    def __str__(self):
        if self.cached is None:
            vars = inspect.currentframe().f_back.f_globals.copy()
            vars.update(inspect.currentframe().f_back.f_locals)
            self.cached = self.payload.format(**vars)
        return self.cached

In [3]:
uuid = str(uuid.uuid4())
item_list_api = "https://apis.zigbang.com/v2/items"
describe_list_api = item_list_api + '/list'
item_describe_api = "https://apis.zigbang.com/v3/items?item_ids={item_id}&detail=true"
item_view_url = "https://zigbang.com/home/villa/items/{item_id}"

referer = "https://zigbang.com/home/villa/subways/414/items"
        
headers = {
    'Host':'apis.zigbang.com',
    'Connection':'keep-alive',
    'Pragma':'no-cache',
    'Cache-Control':'no-cache',
    'Accept':'application/json, text/plain, */*',
    'Origin':'https://zigbang.com',
    'User-Agent':'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Mobile Safari/537.36',
    'DNT':'1',
    'Sec-Fetch-Site':'same-site',
    'Sec-Fetch-Mode':'cors',
    'Referer':'https://zigbang.com/home/villa/subways/414/items',
    'Accept-Encoding':'gzip, deflate, br',
    'Accept-Language':'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
}

items = {}

In [18]:
def export_items(items):
    fieldnames = [
        'address1',
        'address2',
        'address3',
        'agent_address1',
        'agent_email',
        'agent_lat',
        'agent_lng',
        'agent_mobile',
        'agent_name',
        'agent_phone',
        'building_type',
        'deposit',
        'description',
        'elevator',
        'floor',
        'floor_all',
        'images',
        'local1',
        'local2',
        'local3',
        'manage_cost',
        'manage_cost_inc',
        'movein_date',
        'near_subways',
        'options',
        'original_user_phone',
        'parking',
        'pets_text',
        'random_location',
        'room_direction_text',
        'room_type',
        'sales_type',
        'service_type',
        'size',
        'size_m2',
        'title',
        'updated_at2',
        'user_email',
        'user_intro',
        'user_mobile',
        'user_name',
        'user_phone',
        'url'
    ]
    
    filtered_item = [] 
    maximum_updated = datetime.timedelta(days=7)
    

    for item in items:
        item = item['item']

        item_updated = datetime.datetime.strptime(item['updated_at2'], '%Y-%m-%d')
        if maximum_updated < datetime.datetime.now() - item_updated:
            continue
        item_id = item['id']
        url = magic_fstring_function(item_view_url)
        item['url'] = str(url)
        new_item = {}
        for fieldname in fieldnames:
            new_item[fieldname] = item[fieldname]
        filtered_item.append(new_item)
    return filtered_item

In [5]:
def describe_room_list(items):
    item_ids = list(items.keys())
    max_idx = int(len(item_ids) / 30)
    if len(item_ids) % 30 != 0:
        max_idx += 1
    
    items = []
    headers['Accept'] = 'application/json'
    headers['Referer'] = 'https://zigbang.com/home/villa/subways/37/items'
    
    for i in range(max_idx):
        start_idx = i * 30
        end_idx = min((i+1)*30, len(item_ids))
        sub_ids = item_ids[start_idx:end_idx-1]
        item_id = str(sub_ids).replace(' ', '').replace('\'','')
        api = magic_fstring_function(item_describe_api)
        g = api
        resp = requests.get(
            url=str(api),
            headers=headers,
            timeout=5,
        )
        resp.encoding = 'utf-8'
        result = resp.json()
        resp.close()
        items += result['items']
        time.sleep(1)
        
    return items

In [7]:
def get_room_list(
    items,
    domain: str = "zigbang",
    floor_in: str = "ground",
    geohash: str = "wyd",
    rent_gteq: int = 0,
):
    resp = requests.get(
        url=item_list_api,
        params={
            "domain": domain,
            "floor_in": floor_in,
            "geohash": geohash,
            "rent_gteq": rent_gteq,
        },
        headers=headers,
        timeout=5,
    )
    new_items = json.loads(resp.content.decode('utf-8'))
    resp.close()

    for section in new_items['sections']:
        for item in section['item_ids']:
            items[str(item)] = {}

    return items

In [8]:
items = {}

In [9]:
geos_seoul = [
    'wydm'
] 

for geo in geos_seoul:
    items = get_room_list(items, geohash=geo)
    time.sleep(1)

In [13]:
from itertools import islice

len(items)

sliceItem = dict(islice(items.items(), 3000))

len(sliceItem)

3000

In [14]:
print(sliceItem)

{'29236817': {}, '29224598': {}, '29218142': {}, '29220861': {}, '29171781': {}, '29223565': {}, '29162916': {}, '29207068': {}, '29165152': {}, '29205701': {}, '28951513': {}, '29228366': {}, '28827122': {}, '29199552': {}, '28934517': {}, '29050560': {}, '29233354': {}, '29097392': {}, '29067867': {}, '28926659': {}, '29056371': {}, '29223698': {}, '29234135': {}, '29177525': {}, '28878882': {}, '29106636': {}, '29210343': {}, '28987829': {}, '29108580': {}, '29203085': {}, '29213264': {}, '29121574': {}, '29199484': {}, '29189621': {}, '29143511': {}, '29022644': {}, '28972876': {}, '29230611': {}, '29190105': {}, '29058677': {}, '29214198': {}, '29176805': {}, '29202316': {}, '29179672': {}, '29121758': {}, '29183454': {}, '29184502': {}, '29212784': {}, '29193929': {}, '29226536': {}, '29230735': {}, '29134831': {}, '29078052': {}, '29138147': {}, '29141424': {}, '29232615': {}, '28949857': {}, '29237534': {}, '29202250': {}, '29138157': {}, '29198164': {}, '28859854': {}, '292214

In [15]:
result = describe_room_list(sliceItem)

In [16]:
len(result)

2900

In [17]:
pp = pprint.PrettyPrinter(indent=2)
pp.pprint(result[0]['item'])

{ '_floor': '2층',
  'address1': '서울시 광진구 자양동',
  'address2': '666-16',
  'address3': None,
  'agent_address1': '서울특별시 강남구 역삼로64길 20 1층[01호](대치동)',
  'agent_comment': '',
  'agent_email': 'smart6325@naver.com',
  'agent_lat': 37.499738901664095,
  'agent_lng': 127.053110431248,
  'agent_local1': '서울시',
  'agent_local2': '강남구',
  'agent_mobile': '010-6658-6325',
  'agent_name': '강남수부동산공인중개사사무소(권승원)',
  'agent_no': 68758,
  'agent_phone': '02-566-8940',
  'bjd_code': '',
  'bonbun_code': '666',
  'bubun_code': '16',
  'building': None,
  'building_type': '-',
  'contract': '서울특별시',
  'deposit': 33800,
  'description': '〓〓〓〓〓〓〓매물 상세 정보〓〓〓〓〓〓〓\r\n'
                 '\r\n'
                 '▶거실 / 방2 / 화장실1\r\n'
                 '\r\n'
                 '▶시스템에어컨 / 인덕션 / 냉장고 / 공기순환기\r\n'
                 '\r\n'
                 '▶필로티방식 주차장 / CCTV / 무인택배함\r\n'
                 '\r\n'
                 '▶뚝섬 한강공원 3분거리 , 이마트 롯데백화점 인접\r\n'
                 '\r\n'
                 '▶성자초, 광양중,고, 건대사대부속

In [19]:
filtered_result = export_items(result)

In [20]:
len(filtered_result)


1839

In [21]:
def get_full_address(item):
    addr = item['address1']
    addr += ' '
    addr += item['address2'] or ''
    addr += ' '
    addr += item['address3'] or ''
    return addr

In [22]:
def get_location_lat(item):
    locationList = item['random_location'].split(',')

    lat = locationList[0][0:9]
    
    return lat

def get_location_lng(item):
    locationList = item['random_location'].split(',')

    lng = locationList[1][0:10]
    
    return lng

In [23]:
geo_based_dict = {}

In [24]:
for item in filtered_result:
    full_addr = get_full_address(item)
    item['address'] = full_addr
    if full_addr not in geo_based_dict:
        geo_based_dict[full_addr] = []
    geo_based_dict[full_addr].append(item)
    lat = get_location_lat(item)
    item['lat'] = lat
    lng = get_location_lng(item)
    item['lng'] = lng

In [25]:
first_key = next(iter(geo_based_dict))
pp.pprint(geo_based_dict[first_key][0])

{ 'address': '서울시 광진구 자양동 666-16 ',
  'address1': '서울시 광진구 자양동',
  'address2': '666-16',
  'address3': None,
  'agent_address1': '서울특별시 강남구 역삼로64길 20 1층[01호](대치동)',
  'agent_email': 'smart6325@naver.com',
  'agent_lat': 37.499738901664095,
  'agent_lng': 127.053110431248,
  'agent_mobile': '010-6658-6325',
  'agent_name': '강남수부동산공인중개사사무소(권승원)',
  'agent_phone': '02-566-8940',
  'building_type': '-',
  'deposit': 33800,
  'description': '〓〓〓〓〓〓〓매물 상세 정보〓〓〓〓〓〓〓\r\n'
                 '\r\n'
                 '▶거실 / 방2 / 화장실1\r\n'
                 '\r\n'
                 '▶시스템에어컨 / 인덕션 / 냉장고 / 공기순환기\r\n'
                 '\r\n'
                 '▶필로티방식 주차장 / CCTV / 무인택배함\r\n'
                 '\r\n'
                 '▶뚝섬 한강공원 3분거리 , 이마트 롯데백화점 인접\r\n'
                 '\r\n'
                 '▶성자초, 광양중,고, 건대사대부속중,고 등 학세권\r\n'
                 '\r\n'
                 '▶관리비 추후 변동 가능성 있음\r\n'
                 '\r\n'
                 '〓〓〓〓〓〓〓〓〓〓〓〓〓〓〓〓〓〓〓〓〓\n'
                 '21.10.28에 직방에 등록된 

In [26]:
double_list = []
for addr in geo_based_dict.keys():
    double_list.append(geo_based_dict[addr][0])
double_list.sort(key=lambda x: x['size'], reverse=True)

In [27]:
with open('villa.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = [
        'address',
        'address1',
        'address2',
        'address3',
        'agent_address1',
        'agent_email',
        'agent_lat',
        'agent_lng',
        'agent_mobile',
        'agent_name',
        'agent_phone',
        'building_type',
        'deposit',
        'description',
        'elevator',
        'floor',
        'floor_all',
        'images',
        'lat',
        'lng',
        'local1',
        'local2',
        'local3',
        'manage_cost',
        'manage_cost_inc',
        'movein_date',
        'near_subways',
        'options',
        'original_user_phone',
        'parking',
        'pets_text',
        'random_location',
        'room_direction_text',
        'room_type',
        'sales_type',
        'service_type',
        'size',
        'size_m2',
        'title',
        'updated_at2',
        'user_email',
        'user_intro',
        'user_mobile',
        'user_name',
        'user_phone',
        'url'
    ]
    writer = csv.DictWriter(csvfile, extrasaction='ignore', fieldnames=fieldnames)
    writer.writeheader()
    for item in double_list:
        writer.writerow(item)

In [ ]:
import pymysql
import csv

conn = pymysql.connect(host='localhost', user='khweb', password='khWeb@123', db='mybang_db', charset='utf8')
curs = conn.cursor()
sql = "insert into villa (id,deposit,agent_name,agent_mobile,floor,address,size,size_m2,status,room_type,manage_cost_inc,url,upddt) values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s, now())"
f = open('villa.csv', 'r', encoding='utf-8')
rd = csv.reader(f)

#title, description은 이모티콘 인식못하는 에러 -> 일단 제외
for line in rd:
    curs.execute(sql, (line[0], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11], line[12], line[13]))

conn.commit()
conn.close()
f.close()